In [5]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from gensim.models import Word2Vec, KeyedVectors


In [4]:
# Playlists as sentences, ids as words

df = pandas.read_csv('data/1m.csv')
df['track_uri'] = df['track_uri'].apply(lambda x: x.split(':')[-1])

playlists = df.groupby('Unnamed: 0')['track_uri'].apply(list).values

In [9]:
model = Word2Vec(sentences=playlists, vector_size=256, window=100, min_count=1, workers=8)
model.save("word2vec.model")

In [4]:
model = Word2Vec.load("word2vec.model")
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [2]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [3]:
wv['0UaMYEvWZi0ZqiDOoHU3YI']

memmap([ 1.9645484e-03,  2.6790313e-03,  1.2391257e-03, -3.6766578e-03,
         1.5674274e-03, -6.8370684e-04, -3.1618443e-03, -3.1170256e-03,
        -2.4206108e-04,  2.3827399e-03,  1.1525819e-04,  2.1662896e-03,
        -1.1064870e-03,  1.7890381e-03, -2.7921682e-03,  2.1679709e-03,
        -7.9010683e-04, -2.4083115e-03, -6.7070971e-04,  3.5225940e-03,
         6.6608906e-04, -3.5610367e-03, -6.9704553e-04, -2.0075607e-04,
        -2.4209353e-04,  1.3162435e-03,  2.6955416e-03, -3.8722698e-03,
         5.1767792e-04, -2.8624546e-03,  1.9767911e-03,  5.6839359e-05,
         3.8885050e-03,  1.3449575e-03, -1.8881997e-03, -2.5982989e-03,
        -3.8775487e-03,  2.8780403e-03,  2.3001616e-03, -2.9224823e-03,
        -1.6015403e-03, -2.4066595e-03,  1.1386299e-03, -1.0191636e-03,
         2.7642583e-03, -1.1370381e-03, -2.7663549e-03,  1.5281885e-03,
        -1.9111054e-03, -3.8558198e-03,  1.5695029e-03,  3.0659621e-03,
         9.8440354e-04,  5.9464609e-04,  1.2942716e-03,  2.82531

Mappeamos los tracks a vectores usando W2V, siguiendo [el siguiente paper](https://arxiv.org/pdf/2304.12257). Ahora debemos hacer el entrenamiento contrastivo sobre estos vectores, tomando en cuenta su coocurrencia en playlists (como menciona el paper), pero ademas la similitud de sus features de audio.